In [47]:
import xmltodict

def nomes_moedas():
    with open("moedas.xml", "rb") as arquivos_moedas:
        dic_moedas = xmltodict.parse(arquivos_moedas)
        
    moedas = dic_moedas["xml"]
    return moedas

In [48]:
def conversoes_disponiveis():
    with open("conversoes.xml", "rb") as arquivos_conversoes:
        dic_conversoes = xmltodict.parse(arquivos_conversoes)

    conversoes = dic_conversoes["xml"]
    dic_conversoes_disponiveis = {}
    for par_conversao in conversoes:
        moeda_origem, moeda_destino = par_conversao.split("-")
        if moeda_origem in dic_conversoes_disponiveis:
            dic_conversoes_disponiveis[moeda_origem].append(moeda_destino)
        else:
            dic_conversoes_disponiveis[moeda_origem] = [moeda_destino]
    return dic_conversoes_disponiveis
            

In [49]:
import requests

def pegar_cotacao_moeda(moeda_origem, moeda_destino):
    link = f"https://economia.awesomeapi.com.br/last/{moeda_origem}-{moeda_destino}"
    requisicao = requests.get(link)

    cotacao = requisicao.json()[f"{moeda_origem}{moeda_destino}"]["bid"]
    return cotacao


In [50]:
import customtkinter

#criar e configurar a janela
customtkinter.set_appearance_mode("Dark")
customtkinter.set_default_color_theme("dark-blue")
janela = customtkinter.CTk() 
janela.geometry("500x500") #tamanho da janela

dic_conversoes_disponiveis = conversoes_disponiveis()

#criar botoes e titulos
titulo = customtkinter.CTkLabel(janela, text="Conversor de Moedas", font=("Times New Roman", 20))
texto_moeda_origem = customtkinter.CTkLabel(janela, text="Selecione a moeda de origem: ")
texto_moeda_destino = customtkinter.CTkLabel(janela, text="Selecione a moeda de destino: ")

def carregar_moedas_destino(moeda_selecionada):
    lista_moedas_destino = dic_conversoes_disponiveis[moeda_selecionada]
    campo_moeda_destino.configure(values=lista_moedas_destino)
    
campo_moeda_origem = customtkinter.CTkOptionMenu(janela, values=list(dic_conversoes_disponiveis.keys()), 
                                                 command=carregar_moedas_destino)
campo_moeda_destino = customtkinter.CTkOptionMenu(janela, values=["Selecione uma moeda de origem: "])

def converter_moeda():
    moeda_origem = campo_moeda_origem.get()
    moeda_destino = campo_moeda_destino.get()
    if moeda_origem and moeda_destino:
        cotacao = pegar_cotacao_moeda(moeda_origem, moeda_destino)
        texto_cotacao_moeda.configure(text=f"1 {moeda_origem} = {cotacao} {moeda_destino}")

botao_converter = customtkinter.CTkButton(janela, text="Converter", command=converter_moeda)

lista_moedas = customtkinter.CTkScrollableFrame(janela)

texto_cotacao_moeda = customtkinter.CTkLabel(janela, text=" ")

moedas_disponiveis = nomes_moedas()
for codigo_moeda in moedas_disponiveis:
    nome_moeda = moedas_disponiveis[codigo_moeda]
    texto_moeda = customtkinter.CTkLabel(lista_moedas, text=f"{codigo_moeda}: {nome_moeda}")
    texto_moeda.pack()



titulo.pack(padx=10, pady=10)
texto_moeda_origem.pack(padx=10, pady=10)
campo_moeda_origem.pack(padx=10)
texto_moeda_destino.pack(padx=10, pady=10)
campo_moeda_destino.pack(padx=10)
botao_converter.pack(padx=10, pady=10)
texto_cotacao_moeda.pack(padx=10, pady=10)
lista_moedas.pack(padx=10, pady=10)

#loop da janela
janela.mainloop()

